# Getting Tweet Locations

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
# loading in the Xtest file
kincade_fire = pd.read_csv('../datasets/kincade_fire_closed_only.csv')
kincade_fire.head()

,Tweet,closed_probs,closed_open
0,Evacuation Orders - https://socoemergency.org/...,0.665353,1
1,#KincadeFire off John Kincade Road and Burned ...,0.997102,1
2,NEW INCIDENT 4:06 pm Occidental Rd at Frati Ln...,0.998465,1
3,"(Update 50, 10:21 pm) #KincadeFire Air attack ...",0.993657,1
4,Thanks to our electrical crew for getting High...,0.932110,1


In [4]:
pwd

'/Users/wilson_stewart/Documents/Data_Science/DSI/Submissions/Projects/evacuation_routes/tweet_locations'

In [5]:
# reading in the googlemaps_api key from an external file
with open ("../../../../googlemaps_api_key.txt", "r") as myfile:
    api_key = myfile.read().replace('\n', '')

In [6]:
# using googlemaps_places api to find the location the tweet is talking about

def places_search(place):
    places = [] 
    for i in range(len(place.split())):
        if i < (len(place.split())-1):
            places.append(place.split()[i]+'%20')
        else:
            places.append(place.split()[i])
    place_new = ''.join(places)
    url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place_new}&inputtype=textquery&fields=geometry&key='+api_key
    response = requests.get(url)
    resp_json_payload = response.json()
    lat = str(resp_json_payload['candidates'][0]['geometry']['location']['lat'])
    lng = str(resp_json_payload['candidates'][0]['geometry']['location']['lng'])
    return lat+','+lng

In [7]:
# adding the coordinated (if they can be found) for each tweet in the dataframe

def tweet_coords(df):
    location = []
    for tweet in df['Tweet']:
        try:
            coords = places_search(tweet)
            location.append([coords])
        except:
            location.append(np.nan)
    df['location'] = location
    return df

In [8]:
# running above functions
tweet_coords(kincade_fire)

,Tweet,closed_probs,closed_open,location
0,Evacuation Orders - https://socoemergency.org/...,0.665353,1,NaN
1,#KincadeFire off John Kincade Road and Burned ...,0.997102,1,NaN
2,NEW INCIDENT 4:06 pm Occidental Rd at Frati Ln...,0.998465,1,NaN
3,"(Update 50, 10:21 pm) #KincadeFire Air attack ...",0.993657,1,NaN
4,Thanks to our electrical crew for getting High...,0.932110,1,"[38.7301905,-120.7982803]"
...,...,...,...,...
68,We've been through a lot this week. Gun violen...,0.691212,1,NaN
69,"(Update 1, 12:22 am) #NapaRoadIC is declaring ...",0.877711,1,NaN
70,"(Update 56, 4:31 am) #KincadeFire If you haven...",0.980733,1,NaN
71,Anticipated power outages are expected startin...,0.737542,1,NaN


In [9]:
# making sure all values for location are not null
kincade_fire.isnull().sum()

Tweet            0
closed_probs     0
closed_open      0
location        69
dtype: int64

In [10]:
# writing this new dataframe to a csv
kincade_fire.to_csv('../datasets/kincade_fire_closed_coords.csv', index=False)